# Compute the q value for ASE method
The active testing can use these q values. Use uncertainty quantification method: Dropout with 12 runs

In [6]:
import typing
import io
import os

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from scipy.special import softmax

def read_one_results(path):
    with open(path, "r") as outfile:
        data = json.load(outfile)
    return data

def write_one_results(json_data, path):
    with open(path, "w") as outfile:
        json.dump(json_data, outfile)

def np_read(file):
    with open(file, "rb") as outfile:
        data = np.load(outfile)
    return data

def np_write(data, file):
    with open(file, "wb") as outfile:
        np.save(outfile, data)

def check_path_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
base_path = "./pro_data/PSPNet_VOC/val/"
store_path = base_path + "ase"
output_list = ["output" + str(i) for i in range(1,12)]
ego_output_path = base_path + "output"
file_list = os.listdir(base_path + "output")
check_path_exist(store_path)

In [8]:
for file in file_list:
    ego_output = np_read(os.path.join(ego_output_path,file))
    ego_output = softmax(ego_output)
    dropout_output = np.expand_dims(np.copy(ego_output), axis=0)
    for output_path in output_list:
        temp_output = np_read(os.path.join(base_path + output_path,file))
        temp_output = softmax(temp_output)
        dropout_output = np.concatenate((dropout_output, np.expand_dims(np.copy(temp_output), axis=0)), axis=0)
    pi_output = dropout_output.mean(axis=0)
    ase_loss = (-1 * (pi_output * np.log(ego_output))).sum(axis=1)
    np_write(ase_loss, os.path.join(store_path, file))